In [1]:
import numpy
x = numpy.arange(20)
y = numpy.linspace(0, 1, len(x)) ** 2

In [2]:
import toyplot
canvas, axes, mark = toyplot.plot(x, y, width=300)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t669b32c47cb2436c85b49d07026a3459"> <path d="M 50.0 250.0 L 60.0 249.4459833795014 L 70.0 247.7839335180055 L 80.0 245.01385041551248 L 90.0 241.13573407202216 L 100.0 236.1495844875346 L 110.0 230.05540166204986 L 120.0 222.85318559556785 L 130.0 214.54293628808864 L 140.0 205.1246537396122 L 150.0 194.5983379501385 L 160.0 182.96398891966757 L 170.0 170.22160664819947 L 180.0 156.3711911357341 L 190.0 141.4127423822715 L 200.0 125.34626038781163 L 210.0 108.17174515235459 L 220.0 89.88919667590032 L 230.0 70.49861495844878 L 240.0 50.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0 5 10 15 20 0.0 0.5 1.0

In [3]:
canvas, axes, mark = toyplot.plot(x, y, width=300)
axes.x.ticks.locator = toyplot.locator.Extended(only_inside=True)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="te57b6c58409644269ba0877347a85404"> <path d="M 50.0 250.0 L 60.526315789473685 249.4459833795014 L 71.05263157894737 247.7839335180055 L 81.57894736842104 245.01385041551248 L 92.10526315789474 241.13573407202216 L 102.63157894736841 236.1495844875346 L 113.15789473684211 230.05540166204986 L 123.68421052631578 222.85318559556785 L 134.21052631578948 214.54293628808864 L 144.73684210526315 205.1246537396122 L 155.26315789473682 194.5983379501385 L 165.78947368421052 182.96398891966757 L 176.31578947368422 170.22160664819947 L 186.8421052631579 156.3711911357341 L 197.36842105263156 141.4127423822715 L 207.89473684210526 125.34626038781163 L 218.42105263157893 108.17174515235459 L 228.94736842105263 89.88919667590032 L 239.4736842105263 70.49861495844878 L 250.0 50.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0 6 12 18 0.0 0.5 1.0

In [4]:
canvas, axes, mark = toyplot.plot(x, y, width=300)
axes.x.ticks.locator = toyplot.locator.Uniform(count=5)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t8c4453d9a84044529a4b62b049208c3e"> <path d="M 50.0 250.0 L 60.526315789473685 249.4459833795014 L 71.05263157894737 247.7839335180055 L 81.57894736842104 245.01385041551248 L 92.10526315789474 241.13573407202216 L 102.63157894736841 236.1495844875346 L 113.15789473684211 230.05540166204986 L 123.68421052631578 222.85318559556785 L 134.21052631578948 214.54293628808864 L 144.73684210526315 205.1246537396122 L 155.26315789473682 194.5983379501385 L 165.78947368421052 182.96398891966757 L 176.31578947368422 170.22160664819947 L 186.8421052631579 156.3711911357341 L 197.36842105263156 141.4127423822715 L 207.89473684210526 125.34626038781163 L 218.42105263157893 108.17174515235459 L 228.94736842105263 89.88919667590032 L 239.4736842105263 70.49861495844878 L 250.0 50.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0 4.75 9.5 14.25 19 0.0 0.5 1.0

In the meantime, we can also override the default formatting string used to generate the locator labels:

In [5]:
canvas, axes, mark = toyplot.plot(x, y, width=300)
axes.x.ticks.locator = toyplot.locator.Uniform(count=5, format="{:.2f}")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1527d9bf4c584dfbaf8c0f8d481ecedb"> <path d="M 50.0 250.0 L 60.526315789473685 249.4459833795014 L 71.05263157894737 247.7839335180055 L 81.57894736842104 245.01385041551248 L 92.10526315789474 241.13573407202216 L 102.63157894736841 236.1495844875346 L 113.15789473684211 230.05540166204986 L 123.68421052631578 222.85318559556785 L 134.21052631578948 214.54293628808864 L 144.73684210526315 205.1246537396122 L 155.26315789473682 194.5983379501385 L 165.78947368421052 182.96398891966757 L 176.31578947368422 170.22160664819947 L 186.8421052631579 156.3711911357341 L 197.36842105263156 141.4127423822715 L 207.89473684210526 125.34626038781163 L 218.42105263157893 108.17174515235459 L 228.94736842105263 89.88919667590032 L 239.4736842105263 70.49861495844878 L 250.0 50.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0.00 4.75 9.50 14.25 19.00 0.0 0.5 1.0

In [6]:
canvas, axes, mark = toyplot.plot(x, y, xscale="log10", width=300)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1b456a771311451482943f30bada55e4"> <path d="M 50.0 250.0 L 90.0 249.4459833795014 L 114.0823996531185 247.7839335180055 L 128.169700377573 245.01385041551248 L 138.164799306237 241.13573407202216 L 145.9176003468815 236.1495844875346 L 152.25210003069148 230.05540166204986 L 157.60784320114055 222.85318559556785 L 162.24719895935547 214.54293628808864 L 166.339400755146 205.1246537396122 L 170.0 194.5983379501385 L 173.311414812658 182.96398891966757 L 176.33449968380998 170.22160664819947 L 179.11546818454693 156.3711911357341 L 181.69024285425903 141.4127423822715 L 184.0873007244545 125.34626038781163 L 186.32959861247397 108.17174515235459 L 188.4359137102619 89.88919667590032 L 190.4218004082645 70.49861495844878 L 192.3002880762263 50.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0 10 0 10 1 10 2 0.0 0.5 1.0

If you don't like the "superscript" notation that the Log locator produces, you could replace it with your own locator and custom format:

In [7]:
canvas, axes, mark = toyplot.plot(x, y, xscale="log10", width=300)
axes.x.ticks.locator = toyplot.locator.Log(base=10, format="{base}^{exponent}")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td9c433bc58c24c3493040ca3b675ea1e"> <path d="M 50.0 250.0 L 90.0 249.4459833795014 L 114.0823996531185 247.7839335180055 L 128.169700377573 245.01385041551248 L 138.164799306237 241.13573407202216 L 145.9176003468815 236.1495844875346 L 152.25210003069148 230.05540166204986 L 157.60784320114055 222.85318559556785 L 162.24719895935547 214.54293628808864 L 166.339400755146 205.1246537396122 L 170.0 194.5983379501385 L 173.311414812658 182.96398891966757 L 176.33449968380998 170.22160664819947 L 179.11546818454693 156.3711911357341 L 181.69024285425903 141.4127423822715 L 184.0873007244545 125.34626038781163 L 186.32959861247397 108.17174515235459 L 188.4359137102619 89.88919667590032 L 190.4218004082645 70.49861495844878 L 192.3002880762263 50.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0 10^0 10^1 10^2 0.0 0.5 1.0

Or even display raw tick values:

In [8]:
canvas, axes, mark = toyplot.plot(x, y, xscale="log2", width=300)
axes.x.ticks.locator = toyplot.locator.Log(base=2, format="{:.0f}")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t17b32b75918547e3b79ec3eac4ef52fc"> <path d="M 50.0 250.0 L 90.0 249.4459833795014 L 122.0 247.7839335180055 L 140.718800023077 245.01385041551248 L 154.0 241.13573407202216 L 164.3016990363956 236.1495844875346 L 172.718800023077 230.05540166204986 L 179.83535750584335 222.85318559556785 L 186.0 214.54293628808864 L 191.437600046154 205.1246537396122 L 196.30169903639558 194.5983379501385 L 200.7018117963935 182.96398891966757 L 204.71880002307702 170.22160664819947 L 208.41407098051494 156.3711911357341 L 211.83535750584332 141.4127423822715 L 215.0204990594726 125.34626038781163 L 218.0 108.17174515235459 L 220.79881092001082 89.88919667590032 L 223.437600046154 70.49861495844878 L 225.93368043019473 50.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0 1 2 4 8 16 32 0.0 0.5 1.0

In [9]:
numpy.random.seed(1234)
canvas, table = toyplot.matrix(numpy.random.random((5, 5)), width=300)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t5a8c351b906a4380beefd4b583ad7523"> 0 1 2 3 4 0 0.191519 0.622109 0.437728 0.785359 0.779976 1 0.272593 0.276464 0.801872 0.958139 0.875933 2 0.357817 0.500995 0.683463 0.712702 0.370251 3 0.561196 0.503083 0.013768 0.772827 0.882641 4 0.364886 0.615396 0.075381 0.368824 0.933140

By default the Integer locator generates a tick/label for every integer in the range $[0, N)$ ... as you visualize larger matrices, you'll find that a label for every row and column becomes crowded, in which case you can override the default `step` parameter to space-out the labels:

In [10]:
canvas, table = toyplot.matrix(numpy.random.random((50, 50)), width=400, step=5)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="400.0px" viewBox="0 0 400.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t5006b21b1e8744cebd5823a9b8d9af39"> 0 5 10 15 20 25 30 35 40 45 0 0.651378 0.397203 0.788730 0.316836 0.568099 0.869127 0.436173 0.802148 0.143767 0.704261 0.704581 0.218792 0.924868 0.442141 0.909316 0.059809 0.184287 0.047355 0.674881 0.594625 0.533310 0.043324 0.561433 0.329668 0.502967 0.111894 0.607194 0.565945 0.006764 0.617442 0.912123 0.790524 0.992081 0.958802 0.791964 0.285251 0.624917 0.478094 0.195675 0.382317 0.053874 0.451648 0.982005 0.123943 0.119381 0.738523 0.587304 0.471633 0.107127 0.229219 0.899965 0.416754 0.535852 0.006209 0.300642 0.436893 0.612149 0.918198 0.625737 0.705998 0.149834 0.746063 0.831007 0.633726 0.438310 0.152573 0.568410 0.528224 0.951429 0.480359 0.502560 0.536878 0.819202 0.057116 0.669422 0.767117 0.708115 0.796867 0.557761 0.965837 0.147157 0.029647 0.593893 0.114066 0.950810 0.325707 0.193619 0.457812 0.920403 0.879069 0.252616 0.348009 0.182589 0.901796 0.706528 0.726658 0.900088 0.779164 0.599155 0.291125 0.151395 0.335175 0.657552 0.073343 0.055006 0.323195 0.590482 0.853899 0.287062 0.173067 0.134021 0.994654 0.179498 0.317547 0.568291 0.009349 0.900649 0.977241 0.556895 0.084774 0.333002 0.728429 0.142435 0.552469 0.273043 0.974495 0.667787 0.255653 0.108311 0.776181 0.782478 0.761604 0.914403 0.658623 0.568368 0.201756 0.698296 0.952195 0.889963 0.993567 0.818704 0.545122 0.451254 0.890557 0.973265 0.593411 0.366074 0.323095 0.871423 0.215634 0.734945 0.365619 0.801603 0.782736 0.701355 0.622777 0.493683 0.840538 0.712097 0.443909 0.031035 0.363240 0.730722 0.475567 0.344417 0.640880 0.126205 0.171465 0.737086 0.127029 0.369650 0.604334 0.103104 0.802374 0.945553 0.979039 0.881232 0.627682 0.930487 0.724790 0.716678 0.041079 0.439482 0.282070 0.334996 0.083527 0.760849 0.509272 0.661047 0.630314 0.370927 0.446740 0.415108 0.480389 0.983324 0.373424 0.012407 0.921903 0.873276 0.351747 0.630133 0.357827 0.212820 0.223319 0.419564 0.072910 0.650839 0.755686 0.932101 0.376404 0.296927 0.371944 0.827699 0.901140 0.427304 0.002189 0.041965 0.141787 0.598237 0.106461 0.297159 0.058392 0.621325 0.022680 0.855055 0.306337 0.758783 0.563275 0.038164 0.565517 0.658873 0.183266 0.797411 0.612367 0.555653 0.629492 0.686180 0.240383 0.787928 0.856477 0.727796 0.692345 0.472351 0.856873 0.664743 0.333376 0.518145 0.394689 0.031050 0.576840 5 0.850252 0.951489 0.682979 0.076163 0.410078 0.041901 0.174187 0.565402 0.172891 0.566610 0.514004 0.868871 0.720347 0.903591 0.277615 0.256665 0.699703 0.901569 0.983692 0.640913 0.330007 0.606675 0.822160 0.627965 0.117923 0.285878 0.986747 0.431801 0.574234 0.527242 0.197653 0.165708 0.500990 0.990204 0.393992 0.714013 0.175904 0.070696 0.157623 0.161558 0.283637 0.536706 0.773239 0.880344 0.637391 0.852602 0.597955 0.693847 0.537177 0.889230 0.052266 0.783184 0.145173 0.058698 0.058995 0.051971 0.515986 0.404996 0.999650 0.108570 0.327384 0.997830 0.401114 0.883336 0.569139 0.953241 0.885387 0.779286 0.031808 0.962619 0.519635 0.207791 0.875098 0.224151 0.138659 0.725239 0.973789 0.535482 0.444798 0.022111 0.605988 0.964797 0.967591 0.930029 0.184680 0.622962 0.412963 0.363061 0.036603 0.868149 0.672828 0.087472 0.886966 0.782465 0.317304 0.818314 0.507555 0.021193 0.433522 0.446313 0.238820 0.830246 0.744764 0.586479 0.492868 0.487356 0.266741 0.605011 0.753544 0.270584 0.522303 0.098329 0.713637 0.884041 0.567054 0.994482 0.178740 0.012200 0.456998 0.931752 0.846025 0.473330 0.902555 0.225996 0.304154 0.714994 0.724091 0.018676 0.285813 0.580486 0

In [11]:
fruits = ["Apples", "Oranges", "Kiwi", "Miracle Fruit", "Durian"]
counts = [452, 347, 67, 21, 5]

canvas, axes, mark = toyplot.bars(counts, width=400, height=300)
axes.x.ticks.locator = toyplot.locator.Explicit(labels=fruits)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="300.0px" viewBox="0 0 400.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t040ed8a365934a1d9c3b38cf77ea5211"> Apples Oranges Kiwi Miracle Fruit Durian 0 100 200 300 400

Note that in the above example the implicit $[0, N)$ tick locations match the implicit $[0, N)$ X coordinates that are generated for each bar when you don't supply any X coordinates of your own.  This is by design!

You can also use Explicit locators with a list of tick locations, and a set of tick labels will be generated using a format string.  For example:

In [12]:
x = numpy.linspace(0, 2 * numpy.pi)
y = numpy.sin(x)
locations=[0, numpy.pi/2, numpy.pi, 3*numpy.pi/2, 2*numpy.pi]

canvas, axes, mark = toyplot.plot(x, y, width=500, height=300)
axes.x.ticks.locator = toyplot.locator.Explicit(locations=locations, format="{:.2f}")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="300.0px" viewBox="0 0 500.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t75a5b56a2f2940e8bbce06d635770914"> <path d="M 50.0 150.0 L 58.16326530612245 137.2122838315494 L 66.3265306122449 124.63454160904925 L 74.48979591836735 112.4732995120626 L 82.65306122448979 100.92824479960623 L 90.81632653061224 90.1889469508784 L 98.9795918367347 80.43174493965135 L 107.14285714285714 71.81685175319703 L 115.30612244897958 64.4857236994654 L 123.46938775510203 58.55873769841877 L 131.6326530612245 54.133214696333944 L 139.79591836734693 51.28182165855499 L 147.9591836734694 50.05137837993121 L 156.1224489795918 50.46208870508018 L 164.28571428571428 52.50720878181765 L 172.44897959183675 56.15315779502397 L 180.61224489795916 61.34006936269998 L 188.77551020408163 67.98277454030439 L 196.93877551020407 75.97220029246843 L 205.1020408163265 85.17716046922112 L 213.26530612244898 95.4465098789451 L 221.4285714285714 106.61162608824417 L 229.59183673469386 118.48917819763791 L 237.7551020408163 130.88413712986275 L 245.91836734693877 143.59297800192866 L 254.08163265306123 156.40702199807123 L 262.2448979591836 169.1158628701372 L 270.4081632653061 181.51082180236202 L 278.57142857142856 193.3883739117558 L 286.734693877551 204.55349012105484 L 294.8979591836735 214.82283953077882 L 303.0612244897959 224.02779970753153 L 311.2244897959183 232.01722545969557 L 319.3877551020408 238.65993063730002 L 327.55102040816325 243.84684220497604 L 335.7142857142857 247.49279121818236 L 343.87755102040813 249.53791129491984 L 352.0408163265306 249.9486216200688 L 360.204081632653 248.71817834144503 L 368.36734693877554 245.86678530366606 L 376.53061224489795 241.44126230158125 L 384.6938775510204 235.51427630053465 L 392.85714285714283 228.183148246803 L 401.0204081632653 219.5682550603487 L 409.1836734693877 209.81105304912163 L 417.3469387755101 199.07175520039385 L 425.5102040816326 187.5267004879375 L 433.673469387755 175.36545839095083 L 441.83673469387753 162.7877161684507 L 450.0 150.00000000000003" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0.00 1.57 3.14 4.71 6.28 -1.0 -0.5 0.0 0.5 1.0

Finally, you can supply both locations and labels to an Explicit locator:

In [13]:
labels = ["0", u"\u03c0 / 2", u"\u03c0", u"3\u03c0 / 2", u"2\u03c0"]

canvas, axes, mark = toyplot.plot(x, y, width=500, height=300)
axes.x.ticks.locator = toyplot.locator.Explicit(locations=locations, labels=labels)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="300.0px" viewBox="0 0 500.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t66099b480689441cb2a0e3b981937ab1"> <path d="M 50.0 150.0 L 58.16326530612245 137.2122838315494 L 66.3265306122449 124.63454160904925 L 74.48979591836735 112.4732995120626 L 82.65306122448979 100.92824479960623 L 90.81632653061224 90.1889469508784 L 98.9795918367347 80.43174493965135 L 107.14285714285714 71.81685175319703 L 115.30612244897958 64.4857236994654 L 123.46938775510203 58.55873769841877 L 131.6326530612245 54.133214696333944 L 139.79591836734693 51.28182165855499 L 147.9591836734694 50.05137837993121 L 156.1224489795918 50.46208870508018 L 164.28571428571428 52.50720878181765 L 172.44897959183675 56.15315779502397 L 180.61224489795916 61.34006936269998 L 188.77551020408163 67.98277454030439 L 196.93877551020407 75.97220029246843 L 205.1020408163265 85.17716046922112 L 213.26530612244898 95.4465098789451 L 221.4285714285714 106.61162608824417 L 229.59183673469386 118.48917819763791 L 237.7551020408163 130.88413712986275 L 245.91836734693877 143.59297800192866 L 254.08163265306123 156.40702199807123 L 262.2448979591836 169.1158628701372 L 270.4081632653061 181.51082180236202 L 278.57142857142856 193.3883739117558 L 286.734693877551 204.55349012105484 L 294.8979591836735 214.82283953077882 L 303.0612244897959 224.02779970753153 L 311.2244897959183 232.01722545969557 L 319.3877551020408 238.65993063730002 L 327.55102040816325 243.84684220497604 L 335.7142857142857 247.49279121818236 L 343.87755102040813 249.53791129491984 L 352.0408163265306 249.9486216200688 L 360.204081632653 248.71817834144503 L 368.36734693877554 245.86678530366606 L 376.53061224489795 241.44126230158125 L 384.6938775510204 235.51427630053465 L 392.85714285714283 228.183148246803 L 401.0204081632653 219.5682550603487 L 409.1836734693877 209.81105304912163 L 417.3469387755101 199.07175520039385 L 425.5102040816326 187.5267004879375 L 433.673469387755 175.36545839095083 L 441.83673469387753 162.7877161684507 L 450.0 150.00000000000003" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0 π / 2 π 3π / 2 2π -1.0 -0.5 0.0 0.5 1.0

In [14]:
import toyplot.data
data = toyplot.data.commute()
data[:6]

Datetime,Name,Value,Units
2014-05-01T14:08:53.587607Z,Status Since DTC Cleared,0011110100110,
2014-05-01T14:08:53.656972Z,Fuel System Status,0000,
2014-05-01T14:08:53.726403Z,Calculated Load Value,0.0,%
2014-05-01T14:08:53.734393Z,Coolant Temperature,13,C
2014-05-01T14:08:53.804349Z,Short Term Fuel Trim,0.0,%
2014-05-01T14:08:53.873862Z,Long Term Fuel Trim,-3.125,%


Our first step will be to convert the string datetimes from the file into true numeric timestamps.  Note that for this example we're using [Arrow](http://arrow.readthedocs.io), a library that improves upon the builtin datetime functionality in Python.  Note that Arrow is used in the timestamp locator implementation, so it must already be installed for the following examples to work:

In [15]:
import arrow
timestamps = numpy.array([arrow.get(datetime).timestamp() for datetime in data["Datetime"]])

Now, we can plot the data using the timestamps as our independent variable:

In [16]:
observations = numpy.logical_and(data["Name"] == "Vehicle Speed", data["Value"] != "NODATA")
x = timestamps[observations]
y = data["Value"][observations]

In [17]:
canvas, axes, mark = toyplot.plot(
    x, y, label="Vehicle Speed", xlabel="Time", ylabel="km/h",
    width=600, height=300)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="300.0px" viewBox="0 0 600.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tfd2dfb1d41b24a9e9326bcc0852472ed"> <path d="M 105.67064201831818 250.0 L 106.7482896645864 250.0 L 107.79025065898895 250.0 L 108.83241148789725 250.0 L 109.87407684326172 250.0 L 110.91577351093292 250.0 L 111.95763667424521 250.0 L 112.99942950407663 250.0 L 114.04132151603699 250.0 L 115.0834885040919 250.0 L 116.12613598505655 250.0 L 117.16822616259259 250.0 L 118.21003234386444 250.0 L 119.25169666608176 250.0 L 120.2933738231659 250.0 L 121.33511348565419 250.0 L 122.37715582052869 250.0 L 123.41924516359965 250.0 L 124.46099634965262 250.0 L 125.50271634260812 250.0 L 126.54441599051157 250.0 L 127.58630685011545 250.0 L 128.62823084990185 250.0 L 129.67117098967236 250.0 L 130.7130021651586 250.0 L 131.75524131457013 250.0 L 132.79698916276297 250.0 L 133.83870665232342 250.0 L 134.88045934836072 250.0 L 135.92217099666595 250.0 L 136.96412181854248 250.0 L 138.0062961578369 250.0 L 139.04806367556256 250.0 L 140.08976300557455 250.0 L 141.1315050125122 250.0 L 142.17324415842694 250.0 L 143.21516335010529 250.0 L 144.2573430140813 250.0 L 145.29907882213593 250.0 L 146.34090467294058 250.0 L 147.38252850373584 250.0 L 148.42427349090576 250.0 L 149.46598649024963 250.0 L 150.50790015856424 250.0 L 151.5496408144633 250.0 L 152.59119951725006 250.0 L 153.63299115498862 250.0 L 154.6747313340505 250.0 L 155.71642398834229 250.0 L 156.75814000765482 250.0 L 157.79986099402112 250.0 L 158.84156966209412 250.0 L 159.88336483637494 250.0 L 160.92522935072583 250.0 L 161.967435836792 250.0 L 163.0092568397522 250.0 L 164.05109167099 250.0 L 165.09261667728424 250.0 L 166.13429848353067 250.0 L 167.1763044993083 250.0 L 168.2184966802597 238.33333333333331 L 169.2602250178655 230.0 L 170.30195732911426 211.66666666666669 L 171.34366714954376 180.0 L 172.3854074875514 191.66666666666666 L 173.42736685276031 250.0 L 174.46958299477893 250.0 L 175.51129364967343 206.66666666666669 L 176.55320000648499 170.0 L 177.59476681550342 170.0 L 178.6365161736806 218.33333333333331 L 179.67844168345133 211.66666666666669 L 180.72080532709757 173.33333333333334 L 181.7636263370514 150.0 L 182.80544098218283 146.66666666666666 L 183.84718851248425 158.33333333333334 L 184.88982601960498 175.0 L 185.93133715788525 175.0 L 186.9745261669159 160.0 L 188.01614451408386 165.0 L 189.0578809976578 198.33333333333334 L 190.1002681652705 185.0 L 191.14203198750812 161.66666666666669 L 192.18393218517303 163.33333333333331 L 193.22618683179218 158.33333333333334 L 194.26790765921277 151.66666666666666 L 195.30960484345752 145.0 L 196.3513344923655 145.0 L 197.3930601676305 145.0 L 198.4351210196813 143.33333333333334 L 199.4771961768468 140.0 L 200.51910066604614 133.33333333333331 L 201.56064518292746 130.0 L 202.60238734881082 133.33333333333331 L 203.64408965905506 138.33333333333331 L 204.6858743429184 136.66666666666669 L 205.7278310060501 133.33333333333331 L 206.76934568087262 133.33333333333331 L 207.81102367242178 138.33333333333331 L 208.85277167956036 133.33333333333331 L 209.89451400438944 131.66666666666666 L 210.9362321694692 135.0 L 211.97788016001385 141.66666666666669 L 213.01971034208935 151.66666666666666 L 214.06140899658203 190.0 L 215.1031020085017 240.0 L 216.14520100752515 250.0 L 217.18730266888937 240.0 L 218.22899134953818 191.66666666666666 L 219.27072751522064 180.0 L 220.3124624888102 158.33333333333334 L 221.35423815250397 151.66666666666666 L 222.3961551586787 173.33333333333334 L 223.43835866451263 185.0 L 224.4800

As you would expect, the timestamps make very unfriendly tick labels.  Let's use the timestamp locator instead:

In [18]:
canvas, axes, mark = toyplot.plot(
    x, y, label="Vehicle Speed", xlabel="Time", ylabel="km/h",
    width=600, height=300)
axes.x.ticks.show = True
axes.x.ticks.locator = toyplot.locator.Timestamp()

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="300.0px" viewBox="0 0 600.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t55bbfec696a24dfa84518f57a4ce8ef0"> <path d="M 50.0 250.0 L 51.30915486558294 250.0 L 52.57495664253061 250.0 L 53.84100118372163 250.0 L 55.10644381054587 250.0 L 56.37192447638855 250.0 L 57.637607405539995 250.0 L 58.90320489171852 250.0 L 60.16892286697072 250.0 L 61.43497489045599 250.0 L 62.70161062943756 250.0 L 63.96756934147275 250.0 L 65.23318304733428 250.0 L 66.49862441906399 250.0 L 67.76408138292945 250.0 L 69.02961428001319 250.0 L 70.29551487151767 250.0 L 71.56147256982268 250.0 L 72.82701946603758 250.0 L 74.09252846805263 250.0 L 75.35801275436027 250.0 L 76.62372932969934 250.0 L 77.88948616460871 250.0 L 79.15647743324877 250.0 L 80.42212150274302 250.0 L 81.68826118975305 250.0 L 82.95380403104721 250.0 L 84.21930999187171 250.0 L 85.48485872245551 250.0 L 86.75035758716871 250.0 L 88.01614700626266 250.0 L 89.28220796022815 250.0 L 90.547774696591 250.0 L 91.81325859671571 250.0 L 93.07879434189856 250.0 L 94.34432661143505 250.0 L 95.61007760532763 250.0 L 96.87614502785716 250.0 L 98.1416732424729 250.0 L 99.40731084340312 250.0 L 100.67270302508234 250.0 L 101.93824239073015 250.0 L 103.20374289672077 250.0 L 104.46948718068496 250.0 L 105.73502128459037 250.0 L 107.00033434704213 250.0 L 108.26593038503466 250.0 L 109.53146390966567 250.0 L 110.7969396999489 250.0 L 112.06244387467734 250.0 L 113.32795408351406 250.0 L 114.59344932776227 250.0 L 115.8590496620399 250.0 L 117.12473423246878 250.0 L 118.39083423918294 250.0 L 119.65646595082347 250.0 L 120.92211446142142 250.0 L 122.18738658848275 250.0 L 123.45284920027353 250.0 L 124.71870567037845 250.0 L 125.98478829886078 238.33333333333331 L 127.25030743817771 230.0 L 128.51583140478124 211.66666666666669 L 129.7813280489426 180.0 L 131.04686176666505 191.66666666666666 L 132.3126615644252 250.0 L 133.57877330144578 250.0 L 134.84427095933734 206.66666666666669 L 136.11000636109418 170.0 L 137.3753292712106 170.0 L 138.64087394687365 218.33333333333331 L 139.90663261615194 211.66666666666669 L 141.1729235420513 173.33333333333334 L 142.4397700886389 150.0 L 143.7053940766209 146.66666666666666 L 144.97093653173212 158.33333333333334 L 146.23756015422433 175.0 L 147.50281543405544 175.0 L 148.77010903565554 160.0 L 150.03549455567924 165.0 L 151.3010235909337 198.33333333333334 L 152.5673430943698 185.0 L 153.83290534135608 161.66666666666669 L 155.0986332608187 163.33333333333331 L 156.36479177424647 158.33333333333334 L 157.6303017899917 151.66666666666666 L 158.89578308338164 145.0 L 160.16130381570315 145.0 L 161.42681972073805 145.0 L 162.6927428073981 143.33333333333334 L 163.95868327228993 140.0 L 165.22441640522206 133.33333333333331 L 166.48971223426065 130.0 L 167.75524817253495 133.33333333333331 L 169.0207356931246 138.33333333333331 L 170.28632328336562 136.66666666666669 L 171.55211979857086 133.33333333333331 L 172.81737937468702 133.33333333333331 L 174.0828373522827 138.33333333333331 L 175.34838038666828 133.33333333333331 L 176.6139165180341 131.66666666666666 L 177.87942329949726 135.0 L 179.14484483107907 141.66666666666669 L 180.4104876937517 151.66666666666666 L 181.67597077323768 190.0 L 182.94144699797667 240.0 L 184.2074164265881 250.0 L 185.47338908948163 240.0 L 186.7388600524782 191.66666666666666 L 188.00438870154974 180.0 L 189.26991590243534 158.33333333333334 L 190.53549253473568 151.66666666666666 L 191.80124087362066 173.33333333333334 L 193.06733725986982 185.0 L 194.33282907674456 145.0 L 195.5983587395463 125

By default, the timestamp locator chooses a "good" time interval based on the data domain - for this data, it chose to create ticks at five minute intervals.  In addition, the locator also chooses a default format based on the interval - in this case, `month/day hour:minute`.  This is a significant improvement over raw timestamps, but before we continue, we need to address timezone issues.

As alluded to above, in Toyplot **all timestamps must always be UTC timestamps, without exception** ... that is to say that the numeric values to be displayed by the timestamp locator must be the number of seconds since Midnight, January 1st, 1970, UTC time.  Our current example data already contained UTC datetimes (note the "Z" timezone in the data file), so no special conversion was necessary.  If for some reason you're working with datetimes that aren't UTC, you'll need to take their timezone into consideration when converting them to timestamps for display by Toyplot ... and as an aside: you need to *run, not walk* to anyone collecting data using local timestamps, and make them stop!  

Since all Toyplot timestamps are UTC, the times displayed by the timestamp locator are also UTC by default. If you prefer to display times using a specific timezone, you can specify it when creating the locator.

**An Important Note on Timezone Names**

The arrow library (and thus Toyplot) relies on the underlying operating system for its timezone information.  The following examples use "US/Mountain" to identify a timezone, and should work on most posix-like systems including Linux and OSX.  Unfortunately, Windows operating systems use a different naming scheme; if you are using Windows, you will need to substitute a different name, such as "US Mountain Standard Time".

In [19]:
canvas, axes, mark = toyplot.plot(
    x, y, label="Vehicle Speed", xlabel="Time (US/Mountain)", ylabel="km/h",
    width=600, height=300)
axes.x.ticks.show = True
axes.x.ticks.locator = toyplot.locator.Timestamp(timezone="US/Mountain")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="300.0px" viewBox="0 0 600.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t7d8d3361bb684672862bb2ac71f1612c"> <path d="M 50.0 250.0 L 51.30915486558294 250.0 L 52.57495664253061 250.0 L 53.84100118372163 250.0 L 55.10644381054587 250.0 L 56.37192447638855 250.0 L 57.637607405539995 250.0 L 58.90320489171852 250.0 L 60.16892286697072 250.0 L 61.43497489045599 250.0 L 62.70161062943756 250.0 L 63.96756934147275 250.0 L 65.23318304733428 250.0 L 66.49862441906399 250.0 L 67.76408138292945 250.0 L 69.02961428001319 250.0 L 70.29551487151767 250.0 L 71.56147256982268 250.0 L 72.82701946603758 250.0 L 74.09252846805263 250.0 L 75.35801275436027 250.0 L 76.62372932969934 250.0 L 77.88948616460871 250.0 L 79.15647743324877 250.0 L 80.42212150274302 250.0 L 81.68826118975305 250.0 L 82.95380403104721 250.0 L 84.21930999187171 250.0 L 85.48485872245551 250.0 L 86.75035758716871 250.0 L 88.01614700626266 250.0 L 89.28220796022815 250.0 L 90.547774696591 250.0 L 91.81325859671571 250.0 L 93.07879434189856 250.0 L 94.34432661143505 250.0 L 95.61007760532763 250.0 L 96.87614502785716 250.0 L 98.1416732424729 250.0 L 99.40731084340312 250.0 L 100.67270302508234 250.0 L 101.93824239073015 250.0 L 103.20374289672077 250.0 L 104.46948718068496 250.0 L 105.73502128459037 250.0 L 107.00033434704213 250.0 L 108.26593038503466 250.0 L 109.53146390966567 250.0 L 110.7969396999489 250.0 L 112.06244387467734 250.0 L 113.32795408351406 250.0 L 114.59344932776227 250.0 L 115.8590496620399 250.0 L 117.12473423246878 250.0 L 118.39083423918294 250.0 L 119.65646595082347 250.0 L 120.92211446142142 250.0 L 122.18738658848275 250.0 L 123.45284920027353 250.0 L 124.71870567037845 250.0 L 125.98478829886078 238.33333333333331 L 127.25030743817771 230.0 L 128.51583140478124 211.66666666666669 L 129.7813280489426 180.0 L 131.04686176666505 191.66666666666666 L 132.3126615644252 250.0 L 133.57877330144578 250.0 L 134.84427095933734 206.66666666666669 L 136.11000636109418 170.0 L 137.3753292712106 170.0 L 138.64087394687365 218.33333333333331 L 139.90663261615194 211.66666666666669 L 141.1729235420513 173.33333333333334 L 142.4397700886389 150.0 L 143.7053940766209 146.66666666666666 L 144.97093653173212 158.33333333333334 L 146.23756015422433 175.0 L 147.50281543405544 175.0 L 148.77010903565554 160.0 L 150.03549455567924 165.0 L 151.3010235909337 198.33333333333334 L 152.5673430943698 185.0 L 153.83290534135608 161.66666666666669 L 155.0986332608187 163.33333333333331 L 156.36479177424647 158.33333333333334 L 157.6303017899917 151.66666666666666 L 158.89578308338164 145.0 L 160.16130381570315 145.0 L 161.42681972073805 145.0 L 162.6927428073981 143.33333333333334 L 163.95868327228993 140.0 L 165.22441640522206 133.33333333333331 L 166.48971223426065 130.0 L 167.75524817253495 133.33333333333331 L 169.0207356931246 138.33333333333331 L 170.28632328336562 136.66666666666669 L 171.55211979857086 133.33333333333331 L 172.81737937468702 133.33333333333331 L 174.0828373522827 138.33333333333331 L 175.34838038666828 133.33333333333331 L 176.6139165180341 131.66666666666666 L 177.87942329949726 135.0 L 179.14484483107907 141.66666666666669 L 180.4104876937517 151.66666666666666 L 181.67597077323768 190.0 L 182.94144699797667 240.0 L 184.2074164265881 250.0 L 185.47338908948163 240.0 L 186.7388600524782 191.66666666666666 L 188.00438870154974 180.0 L 189.26991590243534 158.33333333333334 L 190.53549253473568 151.66666666666666 L 191.80124087362066 173.33333333333334 L 193.06733725986982 185.0 L 194.33282907674456 145.0 L 195.5983587395463 125

Note that the hour in this data changed from 14:00 to 8:00, which makes more sense, since this data came from a morning commute.  You could also specify a timezone of "utc" (the default) to explicitly document in the code that no timezone conversion is taking place.  Finally, you may use "local" to automatically display the data using whichever timezone is local to the host running the code ... but be careful with this option, since it means that the the content of a figure could change based on where your laptop was when you generated it!

With timezones out of the way, let's focus on the labels.  The current set is a little crowded, so let's look at different ways to clean things up.  First, we can use the `count` attribute to request that the locator choose an interval that produces a specific number of ticks:

In [20]:
canvas, axes, mark = toyplot.plot(
    x, y, label="Vehicle Speed", xlabel="Time (US/Mountain)", ylabel="km/h",
    width=600, height=300)
axes.x.ticks.show = True
axes.x.ticks.locator = toyplot.locator.Timestamp(timezone="US/Mountain", count=5)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="300.0px" viewBox="0 0 600.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t2f3ab4963548497aacca8a90524d8994"> <path d="M 50.0 250.0 L 51.30915486558294 250.0 L 52.57495664253061 250.0 L 53.84100118372163 250.0 L 55.10644381054587 250.0 L 56.37192447638855 250.0 L 57.637607405539995 250.0 L 58.90320489171852 250.0 L 60.16892286697072 250.0 L 61.43497489045599 250.0 L 62.70161062943756 250.0 L 63.96756934147275 250.0 L 65.23318304733428 250.0 L 66.49862441906399 250.0 L 67.76408138292945 250.0 L 69.02961428001319 250.0 L 70.29551487151767 250.0 L 71.56147256982268 250.0 L 72.82701946603758 250.0 L 74.09252846805263 250.0 L 75.35801275436027 250.0 L 76.62372932969934 250.0 L 77.88948616460871 250.0 L 79.15647743324877 250.0 L 80.42212150274302 250.0 L 81.68826118975305 250.0 L 82.95380403104721 250.0 L 84.21930999187171 250.0 L 85.48485872245551 250.0 L 86.75035758716871 250.0 L 88.01614700626266 250.0 L 89.28220796022815 250.0 L 90.547774696591 250.0 L 91.81325859671571 250.0 L 93.07879434189856 250.0 L 94.34432661143505 250.0 L 95.61007760532763 250.0 L 96.87614502785716 250.0 L 98.1416732424729 250.0 L 99.40731084340312 250.0 L 100.67270302508234 250.0 L 101.93824239073015 250.0 L 103.20374289672077 250.0 L 104.46948718068496 250.0 L 105.73502128459037 250.0 L 107.00033434704213 250.0 L 108.26593038503466 250.0 L 109.53146390966567 250.0 L 110.7969396999489 250.0 L 112.06244387467734 250.0 L 113.32795408351406 250.0 L 114.59344932776227 250.0 L 115.8590496620399 250.0 L 117.12473423246878 250.0 L 118.39083423918294 250.0 L 119.65646595082347 250.0 L 120.92211446142142 250.0 L 122.18738658848275 250.0 L 123.45284920027353 250.0 L 124.71870567037845 250.0 L 125.98478829886078 238.33333333333331 L 127.25030743817771 230.0 L 128.51583140478124 211.66666666666669 L 129.7813280489426 180.0 L 131.04686176666505 191.66666666666666 L 132.3126615644252 250.0 L 133.57877330144578 250.0 L 134.84427095933734 206.66666666666669 L 136.11000636109418 170.0 L 137.3753292712106 170.0 L 138.64087394687365 218.33333333333331 L 139.90663261615194 211.66666666666669 L 141.1729235420513 173.33333333333334 L 142.4397700886389 150.0 L 143.7053940766209 146.66666666666666 L 144.97093653173212 158.33333333333334 L 146.23756015422433 175.0 L 147.50281543405544 175.0 L 148.77010903565554 160.0 L 150.03549455567924 165.0 L 151.3010235909337 198.33333333333334 L 152.5673430943698 185.0 L 153.83290534135608 161.66666666666669 L 155.0986332608187 163.33333333333331 L 156.36479177424647 158.33333333333334 L 157.6303017899917 151.66666666666666 L 158.89578308338164 145.0 L 160.16130381570315 145.0 L 161.42681972073805 145.0 L 162.6927428073981 143.33333333333334 L 163.95868327228993 140.0 L 165.22441640522206 133.33333333333331 L 166.48971223426065 130.0 L 167.75524817253495 133.33333333333331 L 169.0207356931246 138.33333333333331 L 170.28632328336562 136.66666666666669 L 171.55211979857086 133.33333333333331 L 172.81737937468702 133.33333333333331 L 174.0828373522827 138.33333333333331 L 175.34838038666828 133.33333333333331 L 176.6139165180341 131.66666666666666 L 177.87942329949726 135.0 L 179.14484483107907 141.66666666666669 L 180.4104876937517 151.66666666666666 L 181.67597077323768 190.0 L 182.94144699797667 240.0 L 184.2074164265881 250.0 L 185.47338908948163 240.0 L 186.7388600524782 191.66666666666666 L 188.00438870154974 180.0 L 189.26991590243534 158.33333333333334 L 190.53549253473568 151.66666666666666 L 191.80124087362066 173.33333333333334 L 193.06733725986982 185.0 L 194.33282907674456 145.0 L 195.5983587395463 125

Note that in most cases the locator won't produce the exact number of ticks requested (we got lucky in this case), it will choose an interval that produces the closest match.

Alternatively, we might choose to accept the default tick count and alter the tick format, to save space:

In [21]:
canvas, axes, mark = toyplot.plot(
    x, y, label="Vehicle Speed", xlabel="Time (US/Mountain)", ylabel="km/h",
    width=600, height=300)
axes.x.ticks.show = True
axes.x.ticks.locator = toyplot.locator.Timestamp(
    timezone="US/Mountain", format="{0:HH}:{0:mm}")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="300.0px" viewBox="0 0 600.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t93d8e50dd4a84068a62e83bebe99f574"> <path d="M 50.0 250.0 L 51.30915486558294 250.0 L 52.57495664253061 250.0 L 53.84100118372163 250.0 L 55.10644381054587 250.0 L 56.37192447638855 250.0 L 57.637607405539995 250.0 L 58.90320489171852 250.0 L 60.16892286697072 250.0 L 61.43497489045599 250.0 L 62.70161062943756 250.0 L 63.96756934147275 250.0 L 65.23318304733428 250.0 L 66.49862441906399 250.0 L 67.76408138292945 250.0 L 69.02961428001319 250.0 L 70.29551487151767 250.0 L 71.56147256982268 250.0 L 72.82701946603758 250.0 L 74.09252846805263 250.0 L 75.35801275436027 250.0 L 76.62372932969934 250.0 L 77.88948616460871 250.0 L 79.15647743324877 250.0 L 80.42212150274302 250.0 L 81.68826118975305 250.0 L 82.95380403104721 250.0 L 84.21930999187171 250.0 L 85.48485872245551 250.0 L 86.75035758716871 250.0 L 88.01614700626266 250.0 L 89.28220796022815 250.0 L 90.547774696591 250.0 L 91.81325859671571 250.0 L 93.07879434189856 250.0 L 94.34432661143505 250.0 L 95.61007760532763 250.0 L 96.87614502785716 250.0 L 98.1416732424729 250.0 L 99.40731084340312 250.0 L 100.67270302508234 250.0 L 101.93824239073015 250.0 L 103.20374289672077 250.0 L 104.46948718068496 250.0 L 105.73502128459037 250.0 L 107.00033434704213 250.0 L 108.26593038503466 250.0 L 109.53146390966567 250.0 L 110.7969396999489 250.0 L 112.06244387467734 250.0 L 113.32795408351406 250.0 L 114.59344932776227 250.0 L 115.8590496620399 250.0 L 117.12473423246878 250.0 L 118.39083423918294 250.0 L 119.65646595082347 250.0 L 120.92211446142142 250.0 L 122.18738658848275 250.0 L 123.45284920027353 250.0 L 124.71870567037845 250.0 L 125.98478829886078 238.33333333333331 L 127.25030743817771 230.0 L 128.51583140478124 211.66666666666669 L 129.7813280489426 180.0 L 131.04686176666505 191.66666666666666 L 132.3126615644252 250.0 L 133.57877330144578 250.0 L 134.84427095933734 206.66666666666669 L 136.11000636109418 170.0 L 137.3753292712106 170.0 L 138.64087394687365 218.33333333333331 L 139.90663261615194 211.66666666666669 L 141.1729235420513 173.33333333333334 L 142.4397700886389 150.0 L 143.7053940766209 146.66666666666666 L 144.97093653173212 158.33333333333334 L 146.23756015422433 175.0 L 147.50281543405544 175.0 L 148.77010903565554 160.0 L 150.03549455567924 165.0 L 151.3010235909337 198.33333333333334 L 152.5673430943698 185.0 L 153.83290534135608 161.66666666666669 L 155.0986332608187 163.33333333333331 L 156.36479177424647 158.33333333333334 L 157.6303017899917 151.66666666666666 L 158.89578308338164 145.0 L 160.16130381570315 145.0 L 161.42681972073805 145.0 L 162.6927428073981 143.33333333333334 L 163.95868327228993 140.0 L 165.22441640522206 133.33333333333331 L 166.48971223426065 130.0 L 167.75524817253495 133.33333333333331 L 169.0207356931246 138.33333333333331 L 170.28632328336562 136.66666666666669 L 171.55211979857086 133.33333333333331 L 172.81737937468702 133.33333333333331 L 174.0828373522827 138.33333333333331 L 175.34838038666828 133.33333333333331 L 176.6139165180341 131.66666666666666 L 177.87942329949726 135.0 L 179.14484483107907 141.66666666666669 L 180.4104876937517 151.66666666666666 L 181.67597077323768 190.0 L 182.94144699797667 240.0 L 184.2074164265881 250.0 L 185.47338908948163 240.0 L 186.7388600524782 191.66666666666666 L 188.00438870154974 180.0 L 189.26991590243534 158.33333333333334 L 190.53549253473568 151.66666666666666 L 191.80124087362066 173.33333333333334 L 193.06733725986982 185.0 L 194.33282907674456 145.0 L 195.5983587395463 125

In [22]:
canvas, axes, mark = toyplot.plot(
    x, y, label="Vehicle Speed", xlabel="Time (US/Mountain)", ylabel="km/h",
    width=600, height=300)
axes.x.ticks.show = True
axes.x.ticks.locator = toyplot.locator.Timestamp(
    timezone="US/Mountain", interval=(7, "minutes"))

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="300.0px" viewBox="0 0 600.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tfa7bc3de165c47cfa130dcf37ea555bd"> <path d="M 50.0 250.0 L 51.30915486558294 250.0 L 52.57495664253061 250.0 L 53.84100118372163 250.0 L 55.10644381054587 250.0 L 56.37192447638855 250.0 L 57.637607405539995 250.0 L 58.90320489171852 250.0 L 60.16892286697072 250.0 L 61.43497489045599 250.0 L 62.70161062943756 250.0 L 63.96756934147275 250.0 L 65.23318304733428 250.0 L 66.49862441906399 250.0 L 67.76408138292945 250.0 L 69.02961428001319 250.0 L 70.29551487151767 250.0 L 71.56147256982268 250.0 L 72.82701946603758 250.0 L 74.09252846805263 250.0 L 75.35801275436027 250.0 L 76.62372932969934 250.0 L 77.88948616460871 250.0 L 79.15647743324877 250.0 L 80.42212150274302 250.0 L 81.68826118975305 250.0 L 82.95380403104721 250.0 L 84.21930999187171 250.0 L 85.48485872245551 250.0 L 86.75035758716871 250.0 L 88.01614700626266 250.0 L 89.28220796022815 250.0 L 90.547774696591 250.0 L 91.81325859671571 250.0 L 93.07879434189856 250.0 L 94.34432661143505 250.0 L 95.61007760532763 250.0 L 96.87614502785716 250.0 L 98.1416732424729 250.0 L 99.40731084340312 250.0 L 100.67270302508234 250.0 L 101.93824239073015 250.0 L 103.20374289672077 250.0 L 104.46948718068496 250.0 L 105.73502128459037 250.0 L 107.00033434704213 250.0 L 108.26593038503466 250.0 L 109.53146390966567 250.0 L 110.7969396999489 250.0 L 112.06244387467734 250.0 L 113.32795408351406 250.0 L 114.59344932776227 250.0 L 115.8590496620399 250.0 L 117.12473423246878 250.0 L 118.39083423918294 250.0 L 119.65646595082347 250.0 L 120.92211446142142 250.0 L 122.18738658848275 250.0 L 123.45284920027353 250.0 L 124.71870567037845 250.0 L 125.98478829886078 238.33333333333331 L 127.25030743817771 230.0 L 128.51583140478124 211.66666666666669 L 129.7813280489426 180.0 L 131.04686176666505 191.66666666666666 L 132.3126615644252 250.0 L 133.57877330144578 250.0 L 134.84427095933734 206.66666666666669 L 136.11000636109418 170.0 L 137.3753292712106 170.0 L 138.64087394687365 218.33333333333331 L 139.90663261615194 211.66666666666669 L 141.1729235420513 173.33333333333334 L 142.4397700886389 150.0 L 143.7053940766209 146.66666666666666 L 144.97093653173212 158.33333333333334 L 146.23756015422433 175.0 L 147.50281543405544 175.0 L 148.77010903565554 160.0 L 150.03549455567924 165.0 L 151.3010235909337 198.33333333333334 L 152.5673430943698 185.0 L 153.83290534135608 161.66666666666669 L 155.0986332608187 163.33333333333331 L 156.36479177424647 158.33333333333334 L 157.6303017899917 151.66666666666666 L 158.89578308338164 145.0 L 160.16130381570315 145.0 L 161.42681972073805 145.0 L 162.6927428073981 143.33333333333334 L 163.95868327228993 140.0 L 165.22441640522206 133.33333333333331 L 166.48971223426065 130.0 L 167.75524817253495 133.33333333333331 L 169.0207356931246 138.33333333333331 L 170.28632328336562 136.66666666666669 L 171.55211979857086 133.33333333333331 L 172.81737937468702 133.33333333333331 L 174.0828373522827 138.33333333333331 L 175.34838038666828 133.33333333333331 L 176.6139165180341 131.66666666666666 L 177.87942329949726 135.0 L 179.14484483107907 141.66666666666669 L 180.4104876937517 151.66666666666666 L 181.67597077323768 190.0 L 182.94144699797667 240.0 L 184.2074164265881 250.0 L 185.47338908948163 240.0 L 186.7388600524782 191.66666666666666 L 188.00438870154974 180.0 L 189.26991590243534 158.33333333333334 L 190.53549253473568 151.66666666666666 L 191.80124087362066 173.33333333333334 L 193.06733725986982 185.0 L 194.33282907674456 145.0 L 195.5983587395463 125

Toyplot supports common intervals in both singular and plural forms from seconds to millennia.

Finally, we might go in the opposite direction and show more detail in the timestamps.  This is often a case where you'll want to adjust the orientation of the labels so they don't overlap (note in the following example that we had to make the canvas larger and position the axes manually on the canvas to make room for the angled labels - and we manually placed the x axis label to avoid overlap):

In [23]:
canvas = toyplot.Canvas(width=600, height=350)
axes = canvas.cartesian(bounds=(80, -80, 50, -120), label="Vehicle Speed", ylabel="km/h")
axes.plot(x, y)
axes.x.ticks.show = True
axes.x.ticks.locator = toyplot.locator.Timestamp(
    timezone="US/Mountain", format="{0:MMMM} {0:d}, {0:YYYY} {0:h}:{0:mm} {0:a}")
axes.x.ticks.labels.angle = 30
canvas.text(300, 320, "Time (US/Mountain)", style={"font-weight":"bold"});

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="350.0px" viewBox="0 0 600.0 350.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td00fd22bca0146bd8eab2ab1c9e98fb7"> <path d="M 80.0 230.0 L 81.15205628171299 230.0 L 82.26596184542693 230.0 L 83.38008104167504 230.0 L 84.49367055328037 230.0 L 85.60729353922193 230.0 L 86.7210945168752 230.0 L 87.8348203047123 230.0 L 88.94865212293422 230.0 L 90.06277790360127 230.0 L 91.17741735390504 230.0 L 92.29146102049602 230.0 L 93.40520108165417 230.0 L 94.51878948877632 230.0 L 95.63239161697791 230.0 L 96.74606056641161 230.0 L 97.86005308693555 230.0 L 98.97409586144396 230.0 L 100.08777713011307 230.0 L 101.2014250518863 230.0 L 102.31505122383703 230.0 L 103.42888181013542 230.0 L 104.54274782485567 230.0 L 105.6577001412589 230.0 L 106.77146692241385 230.0 L 107.88566984698268 230.0 L 108.99934754732155 230.0 L 110.11299279284711 230.0 L 111.22667567576084 230.0 L 112.34031467670846 230.0 L 113.45420936551113 230.0 L 114.56834300500077 230.0 L 115.68204173300009 230.0 L 116.79566756510982 230.0 L 117.90933902087073 230.0 L 119.02300741806285 230.0 L 120.13686829268832 230.0 L 121.2510076245143 230.0 L 122.36467245337616 230.0 L 123.47843354219475 230.0 L 124.59197866207246 230.0 L 125.70565330384252 230.0 L 126.81929374911428 230.0 L 127.93314871900276 230.0 L 129.04681873043953 230.0 L 130.16029422539708 230.0 L 131.2740187388305 230.0 L 132.38768824050578 230.0 L 133.50130693595503 230.0 L 134.61495060971606 230.0 L 135.72859959349236 230.0 L 136.84223540843078 230.0 L 137.9559637025951 230.0 L 139.06976612457254 230.0 L 140.18393413048096 230.0 L 141.29769003672465 230.0 L 142.41146072605085 230.0 L 143.52490019786484 230.0 L 144.63850729624073 230.0 L 145.75246098993304 230.0 L 146.86661370299748 219.5 L 147.9802705455964 212.0 L 149.0939316362075 195.5 L 150.2075686830695 167.0 L 151.32123835466524 177.5 L 152.4351421766942 230.0 L 153.5493205052723 230.0 L 154.66295844421686 191.0 L 155.77680559776286 158.0 L 156.89028975866535 158.0 L 158.0039690732488 201.5 L 159.1178367022137 195.5 L 160.23217271700514 161.0 L 161.34699767800225 140.0 L 162.4607467874264 137.0 L 163.57442414792428 147.50000000000003 L 164.68905293571743 162.5 L 165.8024775819688 162.5 L 166.91769595137686 149.0 L 168.03123520899774 153.5 L 169.14490076002167 183.5 L 170.25926192304541 171.5 L 171.37295670039336 150.5 L 172.48679726952042 152.0 L 173.6010167613369 147.50000000000003 L 174.7146655751927 141.5 L 175.82828911337586 135.5 L 176.94194735781878 135.5 L 178.0556013542495 135.5 L 179.16961367051033 134.0 L 180.28364127961515 131.0 L 181.39748643659541 125.0 L 182.51094676614937 122.0 L 183.62461839183075 125.0 L 184.73824740994968 129.5 L 185.85196448936173 128.0 L 186.96586542274235 125.0 L 188.0792938497246 125.0 L 189.19289687000878 129.5 L 190.3065747402681 125.0 L 191.42024653586998 123.5 L 192.53389250355758 126.50000000000001 L 193.64746345134958 132.5 L 194.76122917050148 141.5 L 195.87485428044914 176.0 L 196.98847335821947 221.0 L 198.10252645539754 230.0 L 199.2165823987438 221.0 L 200.33019684618083 177.5 L 201.4438620573638 167.0 L 202.55752599414308 147.50000000000003 L 203.67123343056744 141.5 L 204.78509196878616 161.0 L 205.89925678868542 171.5 L 207.01288958753523 135.5 L 208.12655569080073 117.5 L 209.24018292475452 111.5 L 210.35386206941752 108.49999999999999 L 211.46774465138537 110.0 L 212.58192476412862 108.49999999999999 L 213.69664263273785 113.0 L 214.81065690308432 114.5 L 215.92435775508972 108.49999999999999 L 217.03800244837367 104.00000000000001 L 218.15167585822024 105.5 L 219.265271784324 